In [190]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
import xml.etree.ElementTree as ET
import keras
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# from keras import losses, metrics, optimizers
from sklearn.metrics import classification_report
import PIL
from PIL import Image


In [191]:
SPLIT_RATIO = 0.2
EPOCH = 7
SYMBOLS = "0123456789-ABCDEFGHIJKLMNOPQRSTUVWXYZ"

class_ids = [symbol for symbol in SYMBOLS]
class_mapping = dict(zip(class_ids, range(len(class_ids))))

path_images = Path("/home/sims/Projects/keras_cv_smbls_only/raw_datasets/ua_plates_dataset/images")
path_annot = Path("/home/sims/Projects/keras_cv_smbls_only/raw_datasets/ua_plates_dataset/annotations")

# mk list of annotation files
xml_files = [
    path_annot.joinpath(file.name) for file in path_annot.iterdir() if file.suffix == ".xml"
]

dataset_x = []
dataset_y = []

# parse annotations and prepare dataset
for xml_file in xml_files:
    plate_no = re.sub(r"^\d\d\d | |_", "", xml_file.stem)
    plate_no = [num for num in plate_no]
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_name = root.find("filename").text
    image_name = path_images.joinpath(image_name)
    
    # make some image preprocessing
    img = Image.open(image_name)
    img = img.convert("L")
    img = PIL.ImageOps.invert(img)
    for obj in root.iter("object"):
        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        smbl_img = img.crop((xmin, ymin, xmax, ymax))
        # size = max(smbl_img.size)
        # bg_img = Image.new(mode="L", size=(size, size), color = 0)
        # bg_img.paste(smbl_img, (0, 0))
        smbl_img = smbl_img.resize((28, 28))
        smbl_img = np.array(smbl_img)
        dataset_x.append(smbl_img)
        dataset_y.append(class_mapping[plate_no.pop(0)])

dataset_x = np.array(dataset_x) / 255
dataset_y = np.array(dataset_y)



In [192]:
num_val = int(len(dataset_x) * SPLIT_RATIO)

val_x = dataset_x[:num_val]
train_x = dataset_x[num_val:]

val_y = dataset_y[:num_val]
train_y = dataset_y[num_val:]

val_y.shape

(5678,)

In [193]:
# data = tf.data.Dataset.from_tensor_slices((dataset_x, dataset_y))

# dataset = data.shuffle(16)
# num_val = int(len(data) * SPLIT_RATIO)

# val_data = data.take(num_val)
# train_data = data.skip(num_val)

# val_x = np.array(tuple(x.numpy() for x, _ in val_data))
# val_y = np.array(tuple(y.numpy() for _, y in val_data))

# train_x = np.array(tuple(x.numpy() for x, _ in train_data))
# train_y = np.array(tuple(y.numpy() for _, y in train_data))


In [194]:
model = keras.Sequential(name="model_2")
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_initializer='he_uniform'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation="relu", kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(len(class_mapping), activation="softmax"))

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

model.summary()

Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 37)             │         2,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,685 (483.14 KB)

 Trainable params: 123,685 (483.14 KB)

 Non-trainable params: 0 (0.00 B)

In [195]:
model.fit(train_x, train_y, epochs=EPOCH, validation_data=(val_x, val_y))

Epoch 1/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.6367 - sparse_categorical_accuracy: 0.8519 - val_loss: 0.0415 - val_sparse_categorical_accuracy: 0.9944
Epoch 2/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0166 - sparse_categorical_accuracy: 0.9969 - val_loss: 0.0335 - val_sparse_categorical_accuracy: 0.9972
Epoch 3/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0123 - sparse_categorical_accuracy: 0.9983 - val_loss: 0.0366 - val_sparse_categorical_accuracy: 0.9967
Epoch 4/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0128 - sparse_categorical_accuracy: 0.9980 - val_loss: 0.0443 - val_sparse_categorical_accuracy: 0.9954
Epoch 5/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0121 - sparse_categorical_accuracy: 0.9973 - val_loss: 0.0444 - val_sparse_categorical_accuracy: 0.9959
Epoch 6/7
710/710 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0076 - sparse_categorical_accuracy: 0.9985 - val_loss: 0.0428 - val_sparse_categorical_accuracy: 0.9965
Epoch 7/7


In [196]:
# print("Classification_report for train dataset")
# print(
#     classification_report(
#         train_y,
#         tf.argmax(model.predict(train_x), axis=1)
#     )
# )
# print("\nClassification_report for test dataset")
# print(
#     classification_report(
#         val_y,
#         tf.argmax(model.predict(val_x), axis=1)
#     )
# )

In [197]:
model.save("mnist_like_v1.keras")